## Neural Networks and Deep Learning

In [21]:
import os

os.environ["CUDA_VISIBLE_DEVICES"] = "-1"

In [22]:
import numpy as np
import matplotlib.pyplot as plt
import tensorflow as tf
from tensorflow.keras import layers, models, optimizers
from tensorflow.keras.preprocessing.image import ImageDataGenerator, load_img, img_to_array, array_to_img

In [23]:
SEED = 42
np.random.seed(SEED)
tf.random.set_seed(SEED)

In [24]:
DATA_URL = "https://github.com/SVizor42/ML_Zoomcamp/releases/download/straight-curly-data/data.zip"
DATA_ZIP = "data.zip"
DATA_DIR = "data"

In [25]:
def download_dataset(url, zip_path):
    if not os.path.exists(zip_path):
        print(f"Baixando o dataset de {url}...")
        os.system(f"wget {url} -O {zip_path}")
    else:
        print(f"O arquivo '{zip_path}' já existe. Nenhuma necessidade de baixar novamente.")

def extract_dataset(zip_path, extract_to):
    if not os.path.exists(extract_to):
        print(f"Extraindo o dataset para '{extract_to}'...")
        os.system(f"unzip {zip_path} -d {extract_to}")
    else:
        print(f"A pasta '{extract_to}' já existe. Nenhuma necessidade de extrair novamente.")

In [26]:
download_dataset(DATA_URL, DATA_ZIP)
extract_dataset(DATA_ZIP, DATA_DIR)

O arquivo 'data.zip' já existe. Nenhuma necessidade de baixar novamente.
A pasta 'data' já existe. Nenhuma necessidade de extrair novamente.
